In [1]:
%autosave 300

Autosaving every 300 seconds


In [2]:
import sys, os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime, time
from timer import date_to_date, time_to_delta

In [3]:
timer_start = datetime.now()

In [4]:
period = ['T1', 'T2', 'T3', 'T4']
path = 'nielsen_data_new_20170328/%s'
dfs = []
for p in period:
    n_log = 0
    print("Loading files from period %s" % p)
    files = os.listdir(path % p)
    for file in files:
        temp_df = pd.read_csv(os.path.join(path % p, file), delimiter = '^', dtype='str', encoding='euc-kr', header=None)
        temp_df['기간'] = p
        n_log += temp_df.shape[0]
        dfs.append(temp_df)
    print("    %s files : %s" % (len(files), n_log))

Loading files from period T1
    14 files : 908408
Loading files from period T2
    14 files : 825801
Loading files from period T3
    14 files : 813856
Loading files from period T4
    14 files : 898153


In [5]:
cols = "일자	패널가구ID	개인ID	가중치	성별	연령	직업\
        학력	소득	채널	시청시작시간	시청종료시간	프로그램시청시간\
        프로그램명	프로그램편성시작시간	프로그램편성종료시간	프로그램장르 기간"
cols = cols.split()

In [22]:
raw_df = pd.concat(dfs, ignore_index=True)
raw_df.columns = cols

In [26]:
channel

,채널,방송사
0,1,EBS
1,2,KBS1
2,3,KBS2
3,4,MBC
4,5,SBS
5,6,AFN
6,7,VCR
7,8,위성기타
8,9,KTV
9,10,어린이TV


In [25]:
raw_df.채널.replace(to_replace=channel)

0            2
1            2
2            2
3            2
4            2
5            2
6            2
7            2
8            2
9            2
10           2
11           2
12           2
13           2
14           3
15           3
16           3
17           3
18           3
19           3
20           3
21           3
22           3
23           3
24           4
25           4
26           4
27           4
28           4
29           4
          ... 
3446188    268
3446189    268
3446190    268
3446191    268
3446192    268
3446193    361
3446194    361
3446195    361
3446196    361
3446197    371
3446198    371
3446199    770
3446200    770
3446201    770
3446202    770
3446203    771
3446204    771
3446205    772
3446206    772
3446207    772
3446208    772
3446209    772
3446210    772
3446211    772
3446212    773
3446213    773
3446214    773
3446215    773
3446216    773
3446217    773
Name: 채널, dtype: object

In [7]:
channel = pd.read_csv('channel.txt', delimiter = '\t', header = 0, dtype='str')
genre = pd.read_csv('genre.txt', delimiter='\t', header = 0, dtype='str')

In [10]:
merged = pd.merge(raw_df, genre, how='left', on='프로그램장르')
merged = pd.merge(merged, channel, how='left', on='채널')

In [9]:
# start = time_to_delta('180000')
# end = time_to_delta('230000')

In [10]:
# sub_df = merged[(merged.시청종료시간.apply(time_to_delta) > start) & (merged.시청시작시간.apply(time_to_delta) < end)]

In [11]:
merged['ID'] = merged.패널가구ID + merged.개인ID

In [12]:
sub_df = merged.ix[:,['기간', '일자', 'ID', '성별', '연령', '직업', '학력', '소득', '방송사', \
                   '시청시작시간', '시청종료시간', '프로그램시청시간', \
                   '프로그램명', '프로그램편성시작시간', '프로그램편성종료시간', '대분류']].dropna(axis = 0)

In [14]:
sub_df['반복'] = sub_df.groupby(['일자', 'ID']).시청시작시간\
                        .apply(lambda t: (t == t.shift(1)) | (t == t.shift(-1)))

In [16]:
tidy_df = sub_df.sort_values(['일자', 'ID', '시청시작시간'])

In [19]:
tidy_df['시청시작시간'] = tidy_df.일자.apply(date_to_date) + tidy_df.시청시작시간.apply(time_to_delta)
tidy_df['시청종료시간'] = tidy_df.일자.apply(date_to_date) + tidy_df.시청종료시간.apply(time_to_delta)
tidy_df['프로그램편성시작시간'] = tidy_df.일자.apply(date_to_date) + tidy_df.프로그램편성시작시간.apply(time_to_delta)
tidy_df['프로그램편성종료시간'] = tidy_df.일자.apply(date_to_date) + tidy_df.프로그램편성종료시간.apply(time_to_delta)

In [20]:
tidy_df['시청시간'] = tidy_df.loc[:,['시청종료시간', '프로그램편성종료시간']].min(axis=1) \
                    - tidy_df.loc[:,['시청시작시간', '프로그램편성시작시간']].max(axis=1) \
                    + timedelta(seconds = 1)

In [21]:
tidy_df[tidy_df.시청시간.apply(timedelta.total_seconds) != tidy_df.프로그램시청시간.apply(int)]

,기간,일자,ID,성별,연령,직업,학력,소득,방송사,시청시작시간,시청종료시간,프로그램시청시간,프로그램명,프로그램편성시작시간,프로그램편성종료시간,대분류,반복,시청시간


In [217]:
tidy_df

,기간,일자,ID,성별,연령,직업,학력,소득,방송사,시청시작시간,시청종료시간,프로그램시청시간,프로그램명,프로그램편성시작시간,프로그램편성종료시간,대분류,반복,시청시간
96,T1,20150817,1401639aa,1,63,1,7,7,O’live,2015-08-17 07:56:00,2015-08-17 08:08:59,780,신동엽성시경은오늘뭐먹지,2015-08-17 07:43:46,2015-08-17 08:12:47,정보,False,00:13:00
0,T1,20150817,1401639aa,1,63,1,7,7,KBS1,2015-08-17 08:09:00,2015-08-17 08:23:59,883,인간극장,2015-08-17 07:50:06,2015-08-17 08:23:42,정보,False,00:14:43
14,T1,20150817,1401639aa,1,63,1,7,7,KBS2,2015-08-17 08:24:00,2015-08-17 08:34:59,660,KBS8아침뉴스타임,2015-08-17 07:57:38,2015-08-17 08:55:04,보도,False,00:11:00
98,T1,20150817,1401639aa,1,63,1,7,7,O’live,2015-08-17 08:35:00,2015-08-17 08:37:59,180,노오븐디저트,2015-08-17 08:18:14,2015-08-17 08:47:53,정보,False,00:03:00
16,T1,20150817,1401639aa,1,63,1,7,7,KBS2,2015-08-17 08:38:00,2015-08-17 08:38:59,60,KBS8아침뉴스타임,2015-08-17 07:57:38,2015-08-17 08:55:04,보도,False,00:01:00
100,T1,20150817,1401639aa,1,63,1,7,7,O’live,2015-08-17 08:39:00,2015-08-17 08:41:59,180,노오븐디저트,2015-08-17 08:18:14,2015-08-17 08:47:53,정보,False,00:03:00
235,T1,20150817,1401639aa,1,63,1,7,7,연합뉴스TV,2015-08-17 08:42:00,2015-08-17 08:52:59,234,라이브투데이1부,2015-08-17 07:54:35,2015-08-17 08:45:53,보도,True,00:03:54
237,T1,20150817,1401639aa,1,63,1,7,7,연합뉴스TV,2015-08-17 08:42:00,2015-08-17 08:52:59,197,라이브투데이2부,2015-08-17 08:49:43,2015-08-17 09:42:01,보도,True,00:03:17
196,T1,20150817,1401639aa,1,63,1,7,7,CHING(채널ING),2015-08-17 08:53:00,2015-08-17 09:05:59,780,마지막황제,2015-08-17 08:39:27,2015-08-17 09:32:36,드라마&영화,False,00:13:00
18,T1,20150817,1401639aa,1,63,1,7,7,KBS2,2015-08-17 09:30:00,2015-08-17 09:30:59,60,TV소설(그래도푸르른날에),2015-08-17 08:58:51,2015-08-17 09:38:55,드라마&영화,False,00:01:00


In [121]:
timer_end = datetime.now()
print(timer_end-timer_start)

0:04:14.408599
